In [37]:
import json
from openai import OpenAI
import os
from neo4j import GraphDatabase
import csv
from datetime import datetime

# Set up the OpenAI API client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
DEPLOYMENT_NAME = 'gpt-4o'

# Example function hard coded to return the same weather


# Function to access Neo4j database
def database_access(cypher_code: str) -> str:
    """Useful for accessing Neo4j graph database via cypher query and returning the result."""
    URI = "bolt://localhost:7687"
    AUTH = ("neo4j", "password")
    
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        with driver.session() as session:
            result = session.run(cypher_code)  # Run Cypher query
            records = list(result)
            return records

def log_message(message_type: str, content: str, query: str = None) -> None:
    """Log message exchanges to CSV file with error handling."""
    LOG_FILE = "chat_history.csv"
    HEADERS = ["Timestamp", "Type", "Content", "Query"]
    
    try:
        # Create file with headers if not exists
        if not os.path.exists(LOG_FILE):
            with open(LOG_FILE, mode='w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(HEADERS)
        
        # Append new message
        with open(LOG_FILE, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([
                datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                message_type,
                content,
                query if query else ""
            ])
    except Exception as e:
        print(f"Error logging message: {e}")

def run_conversation(seed=123):
    messages = [
        {
            "role": "system",
            "content": """
                You are a specialized water system troubleshooting assistant with expertise in analyzing water distribution networks.
                
                You have access to a Neo4j database through cypher queries that contains information about water system components.
                
                Schema details:
                (:TANK)-[:FEEDS_WATER_TO]->(:VALVE)  # Water flow from tanks to valves
                (:VALVE)-[:FEEDS_WATER_TO]->(:VALVE)  # Water flow between valves
                
                When analyzing issues:
                1. Consider only upstream connections if the user query is related to finding the source of a problem
                2. Consider only downstream connections if the user query is related to finding the impact of a problem
                3. Check for 'status' property in the nodes to identify the current state of the components
                4. Consider all possible paths between components for a comprehensive analysis 
                
                Use the database_access function with appropriate cypher queries to investigate the system thoroughly. 

                Conclude the troubleshooting in 2 parts:
                Solution based on database: Directly answer user's query. Conclude as "Unsure" if no information is found.
                Other possible solutions: Use your expertise to provide additional insights or suggestions that were not present in the database.
            """
        }
    ]
    
    while True:
        user_input = input("User: ")
        if user_input.lower() in ['exit', 'quit', 'stop']:
            break
            
        messages.append({"role": "user", "content": user_input})
        log_message("User", user_input)
        
        while True:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=messages,
                tools=[{
                    "type": "function",
                    "function": {
                        "name": "database_access",
                        "description": """Useful for accessing Neo4j graph database via cypher query and returning the result.""",
                        "parameters": {"type": "object", "properties": {"cypher_code": {"type": "string", "description": "The input neo4j cypher code to run in string"}}}
                    }
                }],
                seed=seed,  # Add seed for reproducibility
                tool_choice="auto"
            )
            
            response_message = response.choices[0].message
            log_message("Assistant", response_message.content)
            messages.append(response_message)
            
            if not response_message.tool_calls:
                break
            
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                tool_result = database_access(**function_args)
                log_message("Tool", f"{function_name}: {tool_result}", query=str(function_args))
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": str(tool_result)
                })
                
                # Continue the conversation with the new tool result
                response = client.chat.completions.create(
                    model="gpt-4",
                    messages=messages,
                    seed=seed,
                    temperature=0
                )
                response_message = response.choices[0].message
                log_message("Assistant", response_message.content)
                messages.append(response_message)
        
        print("Assistant:", response_message.content)

# Usage
run_conversation(seed=42)  # Fixed seed for reproducible results


Assistant: Solution based on database: There are no closed valves upstream of 'Valve 2' that could be interrupting the water flow.

Other possible solutions: Given this information, it seems likely that the issue is either a blockage or malfunction in the supply line between 'Tank 1' and 'Valve 2'. A physical inspection of this path should be conducted to verify this. If a blockage is not evident, it may be worthwhile to consider performing maintenance on 'Valve 2' as well, to ensure that it is not malfunctioning in some way not reflected by its "Open" status in the system.
